---
<h1><center> 3. Capstone Data Collecting </center></h1>


--- 

This notebook contains the code used to merge the smaller csv's containing offensive player, defensive player, and shot data into one csv. The objective of having one csv is to have each row of the dataframe containing all data relating to each shot


Note: Towards the end of this notebook there is some code that pertains to when I was jumping between this notebook and number 4 as I caught an error in my data

In [1]:
# packages used
import pandas as pd

In [2]:
#displaying more columns
pd.set_option('display.max_columns', 500)

In [3]:
# loading data
shots1 = pd.read_csv('shot_logs.csv')
print(shots1.shape)

# loading the data
shots2 = pd.read_csv('data/shots_1415.csv')
print(shots2.shape)

(128069, 21)
(198165, 25)


In [4]:
# trying to line up a join
shots1 = pd.get_dummies(shots1, columns=['SHOT_RESULT'])

# renaming column to match other csv
shots1.rename(columns={'player_id': 'PLAYER_ID'}, inplace=True)

# renaming for the join
shots2.rename(columns={'SHOT_MADE_FLAG': 'SHOT_RESULT_made'}, inplace=True)

In [5]:
# splitting the column into minutes and seconds like other csv
shots1['MINUTES_REMAINING'], shots1['SECONDS_REMAINING'] = shots1['GAME_CLOCK'].str.split(':', 1).str

# string to int
shots1['MINUTES_REMAINING'] = pd.to_numeric(shots1['MINUTES_REMAINING'])
shots1['SECONDS_REMAINING'] = pd.to_numeric(shots1['SECONDS_REMAINING'])

# turning into seconds
shots1['TOTAL_TIME_REMAINING'] = (shots1['SECONDS_REMAINING']) + ((shots1['MINUTES_REMAINING']) * 60)

# string to ints
shots2['MINUTES_REMAINING'] = pd.to_numeric(shots2['MINUTES_REMAINING'])
shots2['SECONDS_REMAINING'] = pd.to_numeric(shots2['SECONDS_REMAINING'])

# turning into seconds
shots2['TOTAL_TIME_REMAINING'] = (shots2['SECONDS_REMAINING']) + ((shots2['MINUTES_REMAINING']) * 60)

In [6]:
# trying to join the two DataFrames
shots3 = pd.merge(shots1, shots2, on=['PLAYER_ID','PERIOD','GAME_ID', 'SHOT_RESULT_made'])

# taking the absolute value of the time difference
shots3['DIFF_SECONDS_REMAINING'] = abs(shots3['TOTAL_TIME_REMAINING_x'] - shots3['TOTAL_TIME_REMAINING_y'])

# eliminating duplicates shots created from the poor join
shots_df_1415 = shots3.loc[shots3.groupby(['PLAYER_ID','PERIOD','GAME_ID',\
                          'SHOT_RESULT_made', 'SHOT_NUMBER'])["DIFF_SECONDS_REMAINING"].idxmin()]

print(shots_df_1415.shape)
print(f' There are {shots_df_1415.duplicated().sum()} duplicate rows')

(125794, 48)
 There are 0 duplicate rows


In [7]:
# shots_df_1415.head(10)

In [8]:
# importing player_info
player_info = pd.read_csv('player_info.csv')
print(player_info.shape)
# player_info.head()

(281, 25)


In [9]:
print(shots_df_1415.PLAYER_ID.nunique())
print(player_info.PERSON_ID.nunique())

270
281


In [10]:
# renaming for the join
player_info.rename(columns={'PERSON_ID': 'PLAYER_ID'}, inplace=True)

In [11]:
# joining DataFrames
shots_df_1415 = pd.merge(shots_df_1415, player_info, on=['PLAYER_ID'])
print(shots_df_1415.shape)

(125794, 72)


In [12]:
# shots_df_1415.head(10)

In [13]:
# # defender player info df
# def_player_info = player_info.copy()
# def_player_info.shape

In [14]:
# # renaming for the join
# def_player_info.rename(columns={'PLAYER_ID': 'CLOSEST_DEFENDER_PLAYER_ID'}, inplace=True)

In [15]:
# shots_df_1415 = pd.merge(shots_df_1415, def_player_info, how='left', on=['CLOSEST_DEFENDER_PLAYER_ID'])
# print(shots_df_1415.shape) shape was wrong

In [16]:
# shots_df_1415.head(10)

In [17]:
# saving as a csv
shots_df_1415.to_csv('shots_df_1415.csv')

In [18]:
# columns = ['PLAYER_NAME', 'TEAM_NAME', 'PRIMARY_POSITION', 'HEIGHT', 'WEIGHT',\
#            'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',\
#            'PTS_TYPE', 'SHOT_RESULT_made', 'SHOT_RESULT_missed',\
#            'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',\
#            'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y'\
#            'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST',\ ]

# shots_df_1415

In [19]:
# allPlayerIds = shots_df_1415.copy()

# DefPlayerIds = allPlayerIds.drop_duplicates('CLOSEST_DEFENDER_PLAYER_ID')
# OffPlayerIds = allPlayerIds.drop_duplicates('PLAYER_ID')   

# justDefIds = DefPlayerIds['CLOSEST_DEFENDER_PLAYER_ID'].tolist()
# justOffIds = OffPlayerIds['PLAYER_ID'].tolist()

# print(len(justDefIds))
# print(len(justOffIds))

In [20]:
# c = [i for i in justDefIds if i not in justOffIds]
# print(len(c))

In [21]:
# importing defensive player_info
def_player_info = pd.read_csv('def_player_info.csv')
def_player_info.rename(columns={'PERSON_ID': 'PLAYER_ID'}, inplace=True)
def_player_info.rename(columns={'POSITION': 'PRIMARY_POSITION'}, inplace=True)
print(def_player_info.shape)
print(player_info.shape)
# def_player_info.head()

(202, 25)
(281, 25)


In [22]:
allIds = pd.concat([player_info, def_player_info], ignore_index = True)
allIds.shape

(483, 25)

In [23]:
# allIds.head()

In [24]:
# before i forget i need to find the missing player ids (530027) (203296)

pondexter1 = shots_df_1415.loc[shots_df_1415['CLOSEST_DEFENDER_PLAYER_ID'] == 530027]  



In [25]:
pondexter2 = shots_df_1415.loc[shots_df_1415['CLOSEST_DEFENDER_PLAYER_ID'] == 202347]

In [26]:
print(pondexter1.shape) 
print(pondexter2.shape)

(52, 72)
(278, 72)


### finally found the problem... quincy pondexter was listed twice verified on nba.com that 202347 is the correct id

In [27]:
# confused = shots_df_1415['CLOSEST_DEFENDER_PLAYER_ID'].groupby(shots_df_1415['CLOSEST_DEFENDER']).unique().apply(pd.Series)


In [28]:
# confused.dropna()

In [29]:
allIds.columns

Index(['Unnamed: 0', 'PLAYER_ID', 'FIRST_NAME', 'LAST_NAME',
       'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'PRIMARY_POSITION', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER'],
      dtype='object')

In [30]:
col_list = ['Unnamed: 0', 'PLAYER_ID', 'FIRST_NAME', 'LAST_NAME',\
       'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST',\
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',\
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'PRIMARY_POSITION', 'PLAYERCODE',\
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',\
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER']

quincy = pd.DataFrame(data = [[0, 530027,0,0, 'Quincy Pondexter',\
                       0, 0, 0, 0, 0, 0,'6-7', 230, 0, 20, 3, 0,\
                       0, 0, 0, 0, 0, 2010, 1, 26]], columns = col_list)

quincy.head()

,Unnamed: 0,PLAYER_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,PRIMARY_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
0,0,530027,0,0,Quincy Pondexter,0,0,0,0,0,0,6-7,230,0,20,3,0,0,0,0,0,0,2010,1,26


In [31]:
allIds = allIds.append(quincy, ignore_index = True)
allIds.shape

(484, 25)

In [32]:
# finding 203296
shots_df_1415.loc[shots_df_1415['CLOSEST_DEFENDER_PLAYER_ID'] == 203296]  

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,PLAYER_ID,SHOT_RESULT_made,SHOT_RESULT_missed,MINUTES_REMAINING_x,SECONDS_REMAINING_x,TOTAL_TIME_REMAINING_x,Unnamed: 0_x,GRID_TYPE,GAME_EVENT_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,MINUTES_REMAINING_y,SECONDS_REMAINING_y,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,GAME_DATE,HTM,VTM,TOTAL_TIME_REMAINING_y,DIFF_SECONDS_REMAINING,Unnamed: 0_y,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,PRIMARY_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
7317,21400744,"FEB 05, 2015 - DAL @ SAC",A,W,23,9,4,5:36,10.4,0,2.0,19.1,2,"Miller, Quincy",203296,4.8,0,0,richard jefferson,2210,0,1,5,36,336,98961,Shot Chart Detail,456,Richard Jefferson,1610612742,Dallas Mavericks,5,33,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,17,-164,66,1,20150205,SAC,DAL,333,3,85,Richard,Jefferson,Richard Jefferson,"Jefferson, Richard",R. Jefferson,1980-06-21T00:00:00,Arizona,USA,Arizona/USA,6-7,233,16,22,3,richard_jefferson,2001,2017,N,Y,Y,2001,1,13
19239,21400651,"JAN 23, 2015 - GSW vs. SAC",H,W,25,4,4,5:07,12.7,0,0.9,2.5,2,"Miller, Quincy",203296,3.6,1,2,shaun livingston,2733,1,0,5,7,307,116400,Shot Chart Detail,498,Shaun Livingston,1610612744,Golden State Warriors,5,6,Made Shot,Alley Oop Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,1,1,20150123,GSW,SAC,306,1,34,Shaun,Livingston,Shaun Livingston,"Livingston, Shaun",S. Livingston,1985-09-11T00:00:00,Peoria Central HS (IL),USA,Peoria Central HS (IL)USA/USA,6-7,192,13,34,1,shaun_livingston,2004,2018,Y,Y,Y,2004,1,4
23269,21400730,"FEB 03, 2015 - GSW @ SAC",A,W,25,2,1,3:20,10.7,2,2.6,10.2,2,"Miller, Quincy",203296,1.4,1,2,andrew bogut,101106,1,0,3,20,200,21275,Shot Chart Detail,73,Andrew Bogut,1610612744,Golden State Warriors,3,17,Made Shot,Jump Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,-15,74,1,20150203,SAC,GSW,197,3,35,Andrew,Bogut,Andrew Bogut,"Bogut, Andrew",A. Bogut,1984-11-28T00:00:00,Utah,Australia,Utah/Australia,7-0,260,13,12,5,andrew_bogut,2005,2018,N,Y,Y,2005,1,1
23333,21400651,"JAN 23, 2015 - GSW vs. SAC",H,W,25,4,2,5:49,8.0,0,1.7,7.1,2,"Miller, Quincy",203296,1.9,1,2,andrew bogut,101106,1,0,5,49,349,21253,Shot Chart Detail,207,Andrew Bogut,1610612744,Golden State Warriors,5,47,Made Shot,Jump Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,42,50,1,20150123,GSW,SAC,347,2,35,Andrew,Bogut,Andrew Bogut,"Bogut, Andrew",A. Bogut,1984-11-28T00:00:00,Utah,Australia,Utah/Australia,7-0,260,13,12,5,andrew_bogut,2005,2018,N,Y,Y,2005,1,1
34328,21400607,"JAN 17, 2015 - LAC @ SAC",A,W,9,13,3,1:23,18.9,0,0.8,24.0,3,"Miller, Quincy",203296,9.1,0,0,jj redick,200755,0,1,1,23,83,152680,Shot Chart Detail,390,JJ Redick,1610612746,Los Angeles Clippers,1,20,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,25,212,135,1,20150117,SAC,LAC,80,3,237,JJ,Redick,JJ Redick,"Redick, JJ",J. Redick,1984-06-24T00:00:00,Duke,USA,Duke/USA,6-4,200,12,17,2,jj_redick,2006,2018,N,Y,Y,2006,1,11
37285,21400744,"FEB 05, 2015 - DAL @ SAC",A,W,23,12,4,4:07,21.5,0,1.0,4.9,2,"Miller, Quincy",203296,4.3,1,2,jose juan barea,200826,1,0,4,7,247,11601,Shot Chart Detail,469,J.J. Barea,1610612742,Dallas Mavericks,4,3,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-23,9,1,20150205,SAC,DAL,243,4,79,J.J.,Barea,J.J. Barea,"Barea, J.J.",J. Barea,1984-06-26T00:00:00,Northeastern,Puerto Rico,Northeastern/Puerto Rico,6-0,185,12,5,1,j.j._barea,2006,2018,Y,Y,Y,Undrafted,Undrafted,Undrafted
51938,21400651,"JAN 23, 2015 - GSW vs. SAC",H,W,25,2,1,0:11,4.1,0,

In [33]:
col_list = ['Unnamed: 0', 'PLAYER_ID', 'FIRST_NAME', 'LAST_NAME',\
       'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST',\
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',\
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'PRIMARY_POSITION', 'PLAYERCODE',\
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',\
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER']

quincy_miller = pd.DataFrame(data = [[0, 203296,0,0, 'Quincy Miller',\
                       0, 0, 0, 0, 0, 0,'6-9', 210, 0, 30, 4, 0,\
                       0, 0, 0, 0, 0, 2012, 2, 38]], columns = col_list)

quincy_miller.head()

,Unnamed: 0,PLAYER_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,PRIMARY_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
0,0,203296,0,0,Quincy Miller,0,0,0,0,0,0,6-9,210,0,30,4,0,0,0,0,0,0,2012,2,38


In [34]:
# allIds = allIds.append(quincy, ignore_index = True)
# allIds.shape

In [35]:
allIds = allIds.append(quincy_miller, ignore_index = True)
allIds.shape

(485, 25)

In [36]:
allIds['PLAYER_ID'].nunique()

474

In [37]:
allIds['PLAYER_ID'].value_counts()

2045      2
101131    2
203544    2
201175    2
202366    2
2732      2
200761    2
203133    2
2430      2
201595    2
1890      2
202083    1
203114    1
203108    1
203268    1
203110    1
203106    1
202087    1
203105    1
203112    1
203120    1
203115    1
203118    1
203103    1
203121    1
2419      1
203124    1
2422      1
203128    1
204028    1
         ..
203487    1
203485    1
2744      1
203484    1
2746      1
2747      1
2748      1
2749      1
203457    1
203458    1
203459    1
708       1
2757      1
203462    1
203463    1
2760      1
1737      1
2772      1
203468    1
203469    1
203471    1
202738    1
203473    1
203475    1
203476    1
203477    1
203540    1
203481    1
203482    1
2754      1
Name: PLAYER_ID, Length: 474, dtype: int64

In [38]:
allPlayerIds = allIds.drop_duplicates('PLAYER_ID')

In [39]:
allPlayerIds.shape

(474, 25)

In [40]:
# renaming for join 'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST',\ ]
allPlayerIds.rename(columns={'PLAYER_ID': 'CLOSEST_DEFENDER_PLAYER_ID'}, inplace=True)
allPlayerIds.rename(columns={'DISPLAY_FIRST_LAST': 'CLOSE_DEF_NAME'}, inplace=True)
allPlayerIds.rename(columns={'HEIGHT': 'CLOSE_DEF_HEIGHT'}, inplace=True)
allPlayerIds.rename(columns={'WEIGHT': 'CLOSE_DEF_WEIGHT'}, inplace=True)
allPlayerIds.rename(columns={'PRIMARY_POSITION': 'CLOSE_DEF_POSITION'}, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
allPlayerIds.loc[allPlayerIds['CLOSEST_DEFENDER_PLAYER_ID'] == 530027]

,Unnamed: 0,CLOSEST_DEFENDER_PLAYER_ID,FIRST_NAME,LAST_NAME,CLOSE_DEF_NAME,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,CLOSE_DEF_HEIGHT,CLOSE_DEF_WEIGHT,SEASON_EXP,JERSEY,CLOSE_DEF_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
483,0,530027,0,0,Quincy Pondexter,0,0,0,0,0,0,6-7,230,0,20,3,0,0.0,0.0,0,0,0,2010,1,26


In [42]:
allPlayerIds.drop(483, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [43]:
allPlayerIds.loc[allPlayerIds['CLOSEST_DEFENDER_PLAYER_ID'] == 530027]

,Unnamed: 0,CLOSEST_DEFENDER_PLAYER_ID,FIRST_NAME,LAST_NAME,CLOSE_DEF_NAME,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,CLOSE_DEF_HEIGHT,CLOSE_DEF_WEIGHT,SEASON_EXP,JERSEY,CLOSE_DEF_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER


In [44]:
df = shots_df_1415.merge(allPlayerIds[['CLOSEST_DEFENDER_PLAYER_ID','CLOSE_DEF_NAME', 'CLOSE_DEF_HEIGHT',\
                      'CLOSE_DEF_WEIGHT', 'CLOSE_DEF_POSITION']], on='CLOSEST_DEFENDER_PLAYER_ID')

In [56]:
print(shots_df_1415.shape)
print(df.shape)


(125794, 72)
(125742, 76)


In [46]:
shots_df_1415

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,PLAYER_ID,SHOT_RESULT_made,SHOT_RESULT_missed,MINUTES_REMAINING_x,SECONDS_REMAINING_x,TOTAL_TIME_REMAINING_x,Unnamed: 0_x,GRID_TYPE,GAME_EVENT_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,MINUTES_REMAINING_y,SECONDS_REMAINING_y,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,GAME_DATE,HTM,VTM,TOTAL_TIME_REMAINING_y,DIFF_SECONDS_REMAINING,Unnamed: 0_y,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,PRIMARY_POSITION,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
0,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,1,1,11:37,5.2,0,0.8,15.7,2,"Olynyk, Kelly",203482,3.8,0,0,kevin garnett,708,0,1,11,37,697,63589,Shot Chart Detail,3,Kevin Garnett,1610612751,Brooklyn Nets,11,33,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,72,131,1,20141029,BOS,BKN,693,4,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,1995,1,5
1,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,2,1,8:34,17.6,0,0.8,22.2,2,"Olynyk, Kelly",203482,7.3,1,2,kevin garnett,708,1,0,8,34,514,63590,Shot Chart Detail,35,Kevin Garnett,1610612751,Brooklyn Nets,8,32,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-84,186,1,20141029,BOS,BKN,512,2,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,1995,1,5
2,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,3,1,6:22,1.6,0,0.8,6.7,2,"Olynyk, Kelly",203482,1.7,1,2,kevin garnett,708,1,0,6,22,382,63591,Shot Chart Detail,53,Kevin Garnett,1610612751,Brooklyn Nets,6,19,Made Shot,Fadeaway Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,54,34,1,20141029,BOS,BKN,379,3,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,1995,1,5
3,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,1,1,11:30,9.1,0,0.9,21.3,2,"Smith, Josh",2746,7.3,0,0,kevin garnett,708,0,1,11,30,690,63597,Shot Chart Detail,3,Kevin Garnett,1610612751,Brooklyn Nets,11,27,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,21,102,186,1,20141101,DET,BKN,687,3,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,1995,1,5
4,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,3,1,7:44,12.6,0,1.5,20.3,2,"Jennings, Brandon",201943,14.9,0,0,kevin garnett,708,0,1,7,44,464,63599,Shot Chart Detail,34,Kevin Garnett,1610612751,Brooklyn Nets,7,42,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-97,181,1,20141101,DET,BKN,462,2,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,1995,1,5
5,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,4,1,7:21,13.2,0,0.8,19.1,2,"Caldwell-Pope, Kentavious",203484,5.9,0,0,kevin garnett,708,0,1,7,21,441,63600,Shot Chart Detail,41,Kevin Garnett,1610612751,Brooklyn Nets,7,19,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,17,125,129,1,20141101,DET,BKN,439,2,274,Kevin,Garnett,Kevin Garnett,"Garnett, Kevin",K. Garnett,1976-05-19T00:00:00,Farragut Academy HS (CHI),USA,Farragut Academy HS (IL)/USA,6-11,240,21,21,4,kevin_garnett,1995,2015,N,Y,Y,199

In [47]:
# from preprocessing notebook... checking the nan values in shot_clock
# checking for nans
# First we grab a list of all the columns in our dataframe that have Nans
nan_columns = [col for col in shots_df_1415.columns if shots_df_1415[col].isna().any()]

# Next we look at the total NaNs in each column that has Nans
total_nans_by_column = shots_df_1415[nan_columns].isna().sum().sort_values(ascending=False)
print('Total NaNs by Column:')
print(total_nans_by_column,'\n')

# We can convert these totals to percent by dividing by the total number of values in the dataframe
print('Percent NaNs by Column:')
print(total_nans_by_column/len(shots_df_1415)*100)

Total NaNs by Column:
SHOT_CLOCK    5428
SCHOOL        1902
dtype: int64 

Percent NaNs by Column:
SHOT_CLOCK    4.314991
SCHOOL        1.511996
dtype: float64


In [48]:
# looking at every row in dataframe that contains nan values
nandf = shots_df_1415[shots_df_1415.isna().sum(axis=1)>0] 
# nandf.head(50) 
# after examining can't prove that nan's have anything to do with the the shot clock being very low/ almost expiring

In [49]:
df.rename(columns={'CLOSEST_DEFENDER_PLAYER_ID': 'CLOSE_DEF_ID'}, inplace=True)

In [50]:
df.columns

Index(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN', 'SHOT_NUMBER',
       'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',
       'SHOT_DIST', 'PTS_TYPE', 'CLOSEST_DEFENDER', 'CLOSE_DEF_ID',
       'CLOSE_DEF_DIST', 'FGM', 'PTS', 'player_name', 'PLAYER_ID',
       'SHOT_RESULT_made', 'SHOT_RESULT_missed', 'MINUTES_REMAINING_x',
       'SECONDS_REMAINING_x', 'TOTAL_TIME_REMAINING_x', 'Unnamed: 0_x',
       'GRID_TYPE', 'GAME_EVENT_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME',
       'MINUTES_REMAINING_y', 'SECONDS_REMAINING_y', 'EVENT_TYPE',
       'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'TOTAL_TIME_REMAINING_y', 'DIFF_SECONDS_REMAINING', 'Unnamed: 0_y',
       'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'BIRTHDATE', 'SCHOOL',
       'COUNTRY', 'LAST_AFFILIAT

In [51]:
columns = ['PLAYER_NAME', 'PLAYER_ID', 'TEAM_NAME', 'PRIMARY_POSITION', 'HEIGHT', 'WEIGHT',\
           'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',\
           'PTS_TYPE', 'SHOT_RESULT_made', 'SHOT_RESULT_missed',\
           'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',\
           'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',\
           'CLOSE_DEF_DIST','CLOSE_DEF_NAME', 'CLOSE_DEF_ID',\
           'CLOSE_DEF_HEIGHT', 'CLOSE_DEF_WEIGHT', 'CLOSE_DEF_POSITION']

data = df[columns]
data.shape

(125742, 26)

In [52]:
data.head()

,PLAYER_NAME,PLAYER_ID,TEAM_NAME,PRIMARY_POSITION,HEIGHT,WEIGHT,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,PTS_TYPE,SHOT_RESULT_made,SHOT_RESULT_missed,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,CLOSE_DEF_DIST,CLOSE_DEF_NAME,CLOSE_DEF_ID,CLOSE_DEF_HEIGHT,CLOSE_DEF_WEIGHT,CLOSE_DEF_POSITION
0,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,5.2,0,0.8,2,0,1,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,72,131,3.8,Kelly Olynyk,203482,7-0,240,4
1,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,17.6,0,0.8,2,1,0,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-84,186,7.3,Kelly Olynyk,203482,7-0,240,4
2,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,1.6,0,0.8,2,1,0,Fadeaway Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,54,34,1.7,Kelly Olynyk,203482,7-0,240,4
3,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,10.6,0,0.8,2,0,1,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,17,-146,104,7.3,Kelly Olynyk,203482,7-0,240,4
4,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,14.2,1,2.0,2,0,1,Step Back Jump shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,19,-51,190,3.9,Kelly Olynyk,203482,7-0,240,4


In [53]:
# data.to_csv('capstone_data.csv', index = False)

In [54]:
# jumping back from preprocessing to fix pondexter, and it's a success!
print("Unique Values by Column")
for col in data.columns:
    print(f"{col}: {len(data[col].unique())}")

Unique Values by Column
PLAYER_NAME: 270
PLAYER_ID: 270
TEAM_NAME: 30
PRIMARY_POSITION: 5
HEIGHT: 17
WEIGHT: 62
SHOT_CLOCK: 242
DRIBBLES: 33
TOUCH_TIME: 312
PTS_TYPE: 2
SHOT_RESULT_made: 2
SHOT_RESULT_missed: 2
ACTION_TYPE: 49
SHOT_TYPE: 2
SHOT_ZONE_BASIC: 7
SHOT_ZONE_AREA: 6
SHOT_ZONE_RANGE: 5
SHOT_DISTANCE: 81
LOC_X: 438
LOC_Y: 502
CLOSE_DEF_DIST: 299
CLOSE_DEF_NAME: 473
CLOSE_DEF_ID: 473
CLOSE_DEF_HEIGHT: 17
CLOSE_DEF_WEIGHT: 87
CLOSE_DEF_POSITION: 5
